In [5]:
# Imports using Sklearn make shortcut functions
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder, OneHotEncoder
from sklearn.pipeline import make_pipeline
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from google.cloud import bigquery
from datetime import datetime, timedelta

In [6]:
# Print current time
now = datetime.now()
current_time = now.strftime("%Y-%m-%d %H:%M:%S")

In [7]:
# Imports data
data = pd.read_csv('../processed_data/1988_to_2024_combined_clean_polling_and_results.csv')

In [8]:
# Renames randmom PERCENTAGE COLUMN - FIX IN DATA CLEANING
data.rename(columns={'OTH_PERCENTAGE': 'OTH_ACT'}, inplace=True)

In [9]:
# Convert date columns to datetime
data['enddate'] = pd.to_datetime(data['enddate'])
data['next_elec_date'] = pd.to_datetime(data['next_elec_date'])
data['startdate'] = pd.to_datetime(data['startdate'])

In [10]:
# Handle numerical transformer
num_columns_selector = ['samplesize', 'months_to_elec_weight']
num_transformer = MinMaxScaler()

In [11]:
# Handle categorical transformer
cat_columns_selector = ['rating']
cat_transformer = make_pipeline(OrdinalEncoder(categories = [['F','F+','E-','E','E+','D-','D','D+','C-','C','C+','B-','B','B+','A-']]),MinMaxScaler())

In [12]:
# One-hot encoder for party_in_power
ohe = OneHotEncoder()

In [13]:
# Build the preprocessing pipeline
preproc_pipeline = make_column_transformer(
    (num_transformer, num_columns_selector),
    (cat_transformer, cat_columns_selector),
    (ohe,['party_in_power']),
    remainder='passthrough',
    verbose_feature_names_out=False
)

In [14]:
data['next_elec_date'].value_counts()

next_elec_date
2015-05-07    1930
2024-07-04    1362
2010-05-06     584
1992-04-09     453
2019-12-12     440
1997-05-01     338
2017-06-08     254
2001-06-07     213
2005-05-05      98
Name: count, dtype: int64

In [15]:
# Define election dates
election_date= datetime.strptime('2005-05-05', '%Y-%m-%d')
cutoff_date = election_date - timedelta(days=50)
prediction_date = election_date - timedelta(days=1)

In [16]:
# Split data
data_train = data[data['startdate'] > '2003-12-31']
data_train = data_train[data_train['startdate'] < cutoff_date]
data_test_1 = data[(data['startdate'] >= cutoff_date) & (data['startdate'] < prediction_date)]
data_test = data_test_1[data_test_1['next_elec_date'] == election_date]

In [17]:
# Fit transform preprocessing pipeline to data_train
data_train_processed = preproc_pipeline.fit_transform(data_train)


In [18]:
# Transform preprocessing pipeline to data_test
data_test_processed = preproc_pipeline.transform(data_test)

In [19]:
# Check feature names make sense for data_train
data_train_processed = pd.DataFrame(
    data_train_processed, columns=preproc_pipeline.get_feature_names_out()
)

# Check feature names make sense for data_test
data_test_processed = pd.DataFrame(
    data_test_processed, columns=preproc_pipeline.get_feature_names_out()
)

In [20]:
# Define our X by dropping irrelevant and y columns
X_train = data_train_processed.drop(columns=['startdate', 'enddate', 'pollster', 'Unnamed: 0', 'next_elec_date', 'days_to_elec', 'months_to_elec', 'party_in_power_Labour', 'LAB_ACT', 'CON_ACT', 'LIB_ACT', 'GRE_ACT', 'BRX_ACT', 'SNP_ACT', 'UKI_ACT', 'PLC_ACT', 'OTH_ACT'])
X_test = data_test_processed.drop(columns=['startdate', 'enddate', 'pollster', 'Unnamed: 0', 'next_elec_date', 'days_to_elec', 'months_to_elec', 'party_in_power_Labour', 'LAB_ACT', 'CON_ACT', 'LIB_ACT', 'GRE_ACT', 'BRX_ACT', 'SNP_ACT', 'UKI_ACT', 'PLC_ACT', 'OTH_ACT'])

In [21]:
# Build our target matrix
y_train = data_train_processed[['next_elec_date', 'LAB_ACT', 'CON_ACT', 'LIB_ACT', 'GRE_ACT', 'BRX_ACT', 'SNP_ACT', 'UKI_ACT', 'PLC_ACT', 'OTH_ACT']]
y_test = data_test_processed[['next_elec_date', 'LAB_ACT', 'CON_ACT', 'LIB_ACT', 'GRE_ACT', 'BRX_ACT', 'SNP_ACT', 'UKI_ACT', 'PLC_ACT', 'OTH_ACT']]

In [22]:
# Drops y_train actuals where the actual is the actual result we are trying to predict, replaces with NaNs
y_train.loc[y_train['next_elec_date'] == '2005-05-05',
         ['LAB_ACT', 'CON_ACT', 'LIB_ACT', 'GRE_ACT', 'BRX_ACT', 'SNP_ACT', 'UKI_ACT', 'PLC_ACT', 'OTH_ACT']] = np.nan

In [23]:
y_train

,next_elec_date,LAB_ACT,CON_ACT,LIB_ACT,GRE_ACT,BRX_ACT,SNP_ACT,UKI_ACT,PLC_ACT,OTH_ACT
0,1992-04-09,0.343843,0.419323,0.178082,0.004797,0.0,0.018731,0.0,0.004664,0.030561
1,1992-04-09,0.343843,0.419323,0.178082,0.004797,0.0,0.018731,0.0,0.004664,0.030561
2,1992-04-09,0.343843,0.419323,0.178082,0.004797,0.0,0.018731,0.0,0.004664,0.030561
3,1992-04-09,0.343843,0.419323,0.178082,0.004797,0.0,0.018731,0.0,0.004664,0.030561
4,1992-04-09,0.343843,0.419323,0.178082,0.004797,0.0,0.018731,0.0,0.004664,0.030561
...,...,...,...,...,...,...,...,...,...,...
1075,2005-05-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1076,2005-05-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1077,2005-05-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1078,2005-05-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
X_test.tail(10)

,samplesize,months_to_elec_weight,rating,party_in_power_Conservative,poll_length,CON_FC,LAB_FC,LIB_FC,BRX_FC,GRE_FC,OTH_FC,PLC_FC,SNP_FC,UKI_FC
12,0.175171,0.831387,0.222222,0.0,3,0.31,0.4,0.21,NaN,NaN,0.08,NaN,NaN,NaN
13,0.374014,0.831387,0.222222,0.0,2,0.32,0.4,0.21,NaN,NaN,0.05,NaN,NaN,NaN
14,0.177275,0.831387,0.222222,0.0,3,0.31,0.4,0.21,NaN,NaN,0.08,NaN,NaN,NaN
15,0.180431,0.831387,0.222222,0.0,3,0.31,0.4,0.21,NaN,NaN,0.08,NaN,NaN,NaN
16,0.186744,0.831387,0.222222,0.0,3,0.31,0.4,0.22,NaN,NaN,0.07,NaN,NaN,NaN
17,0.192004,0.831387,0.222222,0.0,3,0.29,0.42,0.21,NaN,NaN,0.08,NaN,NaN,NaN
18,0.193582,1.0,0.222222,0.0,2,0.27,0.41,0.23,NaN,NaN,0.09,NaN,NaN,NaN
19,0.357706,1.0,0.222222,0.0,2,0.32,0.38,0.22,NaN,NaN,0.08,NaN,NaN,NaN
20,0.355602,1.0,0.222222,0.0,1,0.32,0.38,0.21,NaN,NaN,0.08,NaN,NaN,NaN
21,0.350342,1.0,1.0,0.0,1,0.33,0.38,0.23,NaN,NaN,0.06,NaN,NaN,NaN


In [37]:
# Calculates average median value of X_test
averages = X_test.mean()
averages

samplesize                     0.219908
months_to_elec_weight          0.857561
rating                         0.257576
party_in_power_Conservative         0.0
poll_length                    2.454545
CON_FC                         0.320455
LAB_FC                         0.395455
LIB_FC                         0.209545
BRX_FC                              NaN
GRE_FC                              NaN
OTH_FC                         0.073636
PLC_FC                              NaN
SNP_FC                              NaN
UKI_FC                              NaN
dtype: object

In [38]:
# Creates imputation values for y_train to impute over actuals we are trying to predict
imputation_values = {
    'CON_ACT': averages['CON_FC'],
    'LAB_ACT': averages['LAB_FC'],
    'LIB_ACT': averages['LIB_FC'],
    'BRX_ACT': 0, # doesnt exist in 2015
    'GRE_ACT': 0.006,
    'OTH_ACT': averages['OTH_FC'],
    'PLC_ACT':    0.006,
    'SNP_ACT': 0.04, # imputes value due to low forecasts
    'UKI_ACT': 0
}

In [39]:
# Applies imputation values to y_train
y_train = y_train.fillna(value=imputation_values)


In [40]:
# Instantiate the model for each party
models = {}
parties = ['CON', 'LAB', 'LIB', 'BRX', 'GRE', 'SNP', 'UKI', 'PLC', 'OTH']
for party in parties:
    models[party] = XGBRegressor(
        learning_rate=0.3, n_estimators=300, max_depth=3, subsample=0.7,
        objective='reg:squarederror', nthread=-1, enable_categorical=True
    )

In [41]:
# Train models
X_train_matrix = np.array(X_train)
for party in parties:
    models[party].fit(X_train_matrix, y_train[f'{party}_ACT'])

In [31]:
actuals_2005 = {
    "CON": 0.324,  # Conservative Party
    "LAB": 0.352,  # Labour Party
    "LIB": 0.220,  # Liberal Democrats
    "BRX": 0.000,  # Brexit Party (did not exist in 2010)
    "GRE": 0.009,  # Green Party
    "SNP": 0.019,  # Scottish National Party
    "UKI": 0.030,  # UK Independence Party
    "PLC": 0.005,  # Plaid Cymru
    "OTH": 0.048   # Other parties
}

In [32]:
# Evaluate predictions
X_test_matrix = np.array(X_test)
mean_predictions = {}
for party in parties:
    mean_predictions[party] = models[party].predict(X_test_matrix).mean()


In [33]:
# Print mean predictions
for party in parties:
    print(f"2010 mean prediction for {party}: {mean_predictions[party]}")
    print(f"2010 actual result for {party}  : {actuals_2005[party]}")


2010 mean prediction for CON: 0.3201964795589447
2010 actual result for CON  : 0.324
2010 mean prediction for LAB: 0.4021637737751007
2010 actual result for LAB  : 0.352
2010 mean prediction for LIB: 0.20707251131534576
2010 actual result for LIB  : 0.22
2010 mean prediction for BRX: 0.0
2010 actual result for BRX  : 0.0
2010 mean prediction for GRE: 0.006054708268493414
2010 actual result for GRE  : 0.009
2010 mean prediction for SNP: 0.035248707979917526
2010 actual result for SNP  : 0.019
2010 mean prediction for UKI: 0.004235946107655764
2010 actual result for UKI  : 0.03
2010 mean prediction for PLC: 0.006305446848273277
2010 actual result for PLC  : 0.005
2010 mean prediction for OTH: 0.06579572707414627
2010 actual result for OTH  : 0.048


samplesize                     0.219908
months_to_elec_weight          0.857561
rating                         0.257576
party_in_power_Conservative         0.0
poll_length                    2.454545
CON_FC                         0.320455
LAB_FC                         0.395455
LIB_FC                         0.209545
BRX_FC                              NaN
GRE_FC                              NaN
OTH_FC                         0.073636
PLC_FC                              NaN
SNP_FC                              NaN
UKI_FC                              NaN
dtype: object


In [35]:
# Evaluate last five polls
last_predictions = {}
for party in parties:
    last_predictions[party] = models[party].predict(X_test_matrix)[-5:]

In [36]:
# Print mean of predictions
for party in parties:
    print(f"2010 last prediction for {party}: {last_predictions[party].mean()}")
    print(f"2010 actual result for {party}  : {actuals_2005[party]}")


2010 last prediction for CON: 0.3201809823513031
2010 actual result for CON  : 0.324
2010 last prediction for LAB: 0.4052000045776367
2010 actual result for LAB  : 0.352
2010 last prediction for LIB: 0.20559720695018768
2010 actual result for LIB  : 0.22
2010 last prediction for BRX: 0.0
2010 actual result for BRX  : 0.0
2010 last prediction for GRE: 0.006063160486519337
2010 actual result for GRE  : 0.009
2010 last prediction for SNP: 0.03624027594923973
2010 actual result for SNP  : 0.019
2010 last prediction for UKI: 0.0027349742595106363
2010 actual result for UKI  : 0.03
2010 last prediction for PLC: 0.006340638734400272
2010 actual result for PLC  : 0.005
2010 last prediction for OTH: 0.06440185010433197
2010 actual result for OTH  : 0.048
